# Malware Shellcode in Verclsid Target Process
Detects a process access to verclsid.exe that injects shellcode from a Microsoft Office application / VBA macro

## Rule Content
```
- title: Malware Shellcode in Verclsid Target Process
  id: b7967e22-3d7e-409b-9ed5-cdae3f9243a1
  status: experimental
  description: Detects a process access to verclsid.exe that injects shellcode from
    a Microsoft Office application / VBA macro
  references:
  - https://twitter.com/JohnLaTwC/status/837743453039534080
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1055
  author: John Lambert (tech), Florian Roth (rule)
  date: 2017/03/04
  logsource:
    product: windows
    service: sysmon
    definition: 'Use the following config to generate the necessary Event ID 10 Process
      Access events: <ProcessAccess onmatch="include"><CallTrace condition="contains">VBE7.DLL</CallTrace></ProcessAccess><ProcessAccess
      onmatch="exclude"><CallTrace condition="excludes">UNKNOWN</CallTrace></ProcessAccess>'
    category: null
  detection:
    selection:
      EventID: 10
      TargetImage: '*\verclsid.exe'
      GrantedAccess: '0x1FFFFF'
    combination1:
      CallTrace: '*|UNKNOWN(*VBE7.DLL*'
    combination2:
      SourceImage: '*\Microsoft Office\\*'
      CallTrace: '*|UNKNOWN*'
    condition: selection and 1 of combination*
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"10" AND target_process_path.keyword:*\\verclsid.exe AND process_granted_access:"0x1FFFFF") AND (process_call_trace.keyword:*|UNKNOWN\(*VBE7.DLL* OR (process_path.keyword:*\\Microsoft\ Office\\* AND process_call_trace.keyword:*|UNKNOWN*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()